In [17]:

import csv as csv
import numpy as np
import pandas
from ggplot import *
import matplotlib.pyplot as plt
%matplotlib inline
import pylab
from sklearn.preprocessing import LabelEncoder
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold




In [18]:
#load in CSV files


#load in csv file train data
data = pandas.read_csv('train-2.csv', header = 0)

#load in csv file test data
data_test = pandas.read_csv('test-2.csv', header = 0)

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [28]:
labels = data['label']
labels=labels.reshape(-1,1)
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
label2 = pandas.DataFrame(enc.fit_transform(labels).toarray())
data2 = data.drop(['label'], axis=1)


data2 = data2.values.reshape(data2.shape[0],28,28,1)
data_test = data_test.values.reshape(data_test.shape[0],28,28,1)

In [30]:
print data.shape[0]
print data2.shape
print label2.shape
print data_test.shape


42000
(42000, 28, 28, 1)
(42000, 10)
(28000, 28, 28, 1)


In [31]:
#create training and validation datasets

data2= np.array(data2,dtype=np.float32)
labels = np.array(label2,dtype = np.float32)
data_test = np.array(data_test, dtype = np.float32)
train_data, validation_data, train_labels, valid_labels = train_test_split(data2,labels, test_size = 0.15)






In [46]:
#Function
batch_size = 32
patch = 5
depth = 16
num_hidden = 64
import tensorflow as tf

graph=tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, 28,28,1))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, 10))
    tf_valid_dataset = tf.constant(validation_data)
    tf_test_dataset = tf.constant(data_test)
    
    
    #Variables:
    
    weights = {'w1':tf.Variable(tf.truncated_normal([patch, patch,1,depth ], stddev=0.1)),
               'w2':tf.Variable(tf.truncated_normal([patch, patch,depth,depth ], stddev=0.1)),
               'w3':tf.Variable(tf.truncated_normal([28//4*28//4*depth, num_hidden ], stddev=0.1)),
               'out':tf.Variable(tf.truncated_normal([num_hidden,10], stddev=0.1))}
    biases = { 'b1': tf.Variable(tf.zeros([depth])),
              'b2': tf.Variable(tf.constant(1.0,shape=[depth])),
              'b3': tf.Variable(tf.constant(1.0,shape=[num_hidden])),
              'out': tf.Variable(tf.constant(1.0,shape=[10]))}
        
    def model(tf_train_dataset, weights, biases):
        conv = tf.nn.conv2d(tf_train_dataset, weights['w1'],[1,2,2,1],padding='SAME')
        hidden = tf.nn.relu(conv + biases['b1'])
        conv = tf.nn.conv2d(hidden, weights['w2'],[1,2,2,1],padding='SAME')
        hidden = tf.nn.relu(conv + biases['b2'])
        shape = hidden.get_shape().as_list()
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden2 = tf.matmul(reshape, weights['w3'])+biases['b3']
        hidden2 = tf.nn.relu(hidden2)
        logit = tf.matmul(hidden2, weights['out'])+biases['out']
        return logit
        
    #minimize cross entropy
    logit = model(tf_train_dataset, weights, biases)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logit, tf_train_labels))*.01
    
    #optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    #variables for reporting predictions
    train_prediction = tf.nn.softmax(logit)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset, weights, biases))
    test_prediction = tf.nn.softmax(model(tf_test_dataset, weights, biases))


In [ ]:
#definition of accuracy

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions,1)==np.argmax(labels,1))/predictions.shape[0])

results = np.zeros([28000,10])

with tf.Session(graph=graph) as session:
    #initialize variales
    tf.initialize_all_variables().run()
    print('Initialized')
    
    for i in range(10001):
        offset = (batch_size*i)%(train_labels.shape[0] -batch_size)
        batch_data = train_data[offset:offset+batch_size,:,:,:]
        batch_labels = train_labels[offset:offset+batch_size,:]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict = feed_dict)
        if (i % 500 == 0):
            print('Loss at step %d: %f' % (i, l))
            print('Training accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    results = test_prediction.eval()

        
     




Initialized
Loss at step 0: 0.746846
Training accuracy: 6.2%
Validation accuracy: 8.7%
Loss at step 500: 0.007188
Training accuracy: 71.9%
Validation accuracy: 86.3%
Loss at step 1000: 0.004217
Training accuracy: 87.5%
Validation accuracy: 89.5%
Loss at step 1500: 0.004167
Training accuracy: 87.5%
Validation accuracy: 90.7%
Loss at step 2000: 0.001872
Training accuracy: 90.6%
Validation accuracy: 92.0%
Loss at step 2500: 0.001948
Training accuracy: 90.6%
Validation accuracy: 92.4%
Loss at step 3000: 0.005251
Training accuracy: 90.6%
Validation accuracy: 92.6%
Loss at step 3500: 0.004078
Training accuracy: 96.9%
Validation accuracy: 92.6%
Loss at step 4000: 0.001605
Training accuracy: 90.6%
Validation accuracy: 93.2%


In [70]:
results = pandas.DataFrame(results)
results['Index']=results.index +1
results['Final']=0
#results['Max']=0
#results.columns = ['00','01','02','03','04','05','06','07','08','09','Index','Final']
results['Final']=results[[0,1,2,3,4,5,6,7,8,9]].idxmax(axis = 1)
print results

#results.loc[(results[0]==),'Final']=0
#results.loc[(results[1]==1),'Final']=1
#results.loc[(results[2]==1),'Final']=2
#results.loc[(results[3]==1),'Final']=3
#results.loc[(results[4]==1),'Final']=4
#results.loc[(results[5]==1),'Final']=5
#results.loc[(results[6]==1),'Final']=6
#results.loc[(results[7]==1),'Final']=7
#results.loc[(results[8]==1),'Final']=8
#results.loc[(results[9]==1),'Final']=9


                  0         1         2         3         4         5  \
0      4.234096e-03  0.000249  0.962885  0.002930  0.000923  0.001854   
1      7.336589e-01  0.050019  0.016445  0.004117  0.020152  0.037291   
2      1.799937e-04  0.027653  0.001401  0.000226  0.255423  0.000735   
3      2.511209e-02  0.006296  0.067748  0.005250  0.172719  0.007763   
4      4.170848e-04  0.000042  0.008882  0.977106  0.000039  0.006240   
5      9.085230e-05  0.000577  0.009330  0.078797  0.002662  0.003370   
6      7.682786e-01  0.032688  0.016517  0.011875  0.007420  0.039936   
7      1.523377e-03  0.000275  0.016687  0.860037  0.001518  0.077824   
8      7.023046e-01  0.016516  0.088684  0.057157  0.000805  0.018195   
9      1.211508e-03  0.000176  0.006137  0.904028  0.000411  0.059580   
10     3.264086e-03  0.040435  0.009192  0.051175  0.024699  0.124998   
11     3.244230e-07  0.000034  0.022560  0.000506  0.000056  0.000083   
12     1.832125e-03  0.007912  0.080185  0.111636  

In [71]:

results.columns = ['00','01','02','03','04','05','06','07','08','09','Index','Final']
results = results.drop(['00','01','02','03','04','05','06','07','08','09'],axis=1)


print results.describe()



              Index         Final
count  28000.000000  28000.000000
mean   14000.500000      4.645143
std     8083.048105      2.874993
min        1.000000      0.000000
25%     7000.750000      2.000000
50%    14000.500000      5.000000
75%    21000.250000      7.000000
max    28000.000000      9.000000


In [50]:
np.savetxt("results.csv", results, delimiter = ',',fmt = '%.1i',comments='',header = "ImageID,Label")

